In [2]:
'''    imports    '''
# -*- coding: utf-8 -*-

import numpy as np
import sklearn
import csv
import re

from time import time
from pandas import read_csv

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report

In [165]:
'''    functions    '''
def printEach(list):
    for item in list:
        print(item)
        
def printBestParameter(grid_search):
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

def translateNumbers(x):
    # translate indian numbers to arabic numbers
    x = x.replace(u'\u0660','0')
    x = x.replace(u'\u0661','1')
    x = x.replace(u'\u0662','2')
    x = x.replace(u'\u0663','3')
    x = x.replace(u'\u0664','4')
    x = x.replace(u'\u0665','5')
    x = x.replace(u'\u0666','6')
    x = x.replace(u'\u0667','7')
    x = x.replace(u'\u0668','8')
    x = x.replace(u'\u0669','9')
    return x
    
def removePunc(x):    
    # removes punctuation and arabic harakat
    punc = """!$%^&*()-=+.,:'"<>/\?"""
    arabic_semicolon = u"\u061B"
    arabic_comma = u"\u060C"
    arabic_question = u"\u061F"
    arabia_fatha = u"\u064E"
    arabia_2fathas = u"\u064B"
    arabia_damma = u"\u064F"
    arabia_2dammas = u"\u064C"
    arabia_kasra = u"\u0650"
    arabia_2kasras = u"\u064D"
    arabic_sukun = u"\u0652"
    arabic_shadda = u"\u0651"
    punc += arabic_semicolon + arabic_comma + arabic_question + arabia_fatha + arabia_2fathas + arabia_damma + arabia_2dammas + arabia_kasra + arabia_2kasras + arabic_sukun + arabic_shadda
    
    for char in punc:
        x = x.replace(char, "")
        
    return x

def uniformArabic(x):
    # uniform multiple characters to thier origrinal form
    # Alifs -> Alif
    x = x.replace(u'\u0622',u'\u0627')
    x = x.replace(u'\u0623',u'\u0627')
    x = x.replace(u'\u0625',u'\u0627')
    # Taa marbotah -> haa
    x = x.replace(u"\u0629",u"\u0647")
    # Alif_maqsurah + Yaa_Hamzah -> Yaa
    x = x.replace(u'\u0649',u'\u064A')
    x = x.replace(u'\u0626',u'\u064A')
    # Waw_hamzah -> waw
    x = x.replace(u'\u0624',u'\u0648')
    return x

def reduceString(x, string):
    # replaces two same consecutive characters with one
    stringString = string+string
    while stringString in x:
        x = x.replace(stringString, string)
    return x
    
def clean(x):
    # cleans the tweet
    x = translateNumbers(x)
    
    x = re.sub("(https|http)://t.co/([a-zA-Z0-9]){10}"," _LINK_ ",x) # Link
    x = re.sub("[0-9]{10,}"," _NUMBER_ ",x) # Number
    x = re.sub("@.*\s"," _ACCOUNT_ ",x) # Account
    
    x = removePunc(x)
    
    x = uniformArabic(x)
    
    x = reduceString(x, ' ')
    
    # reduce arabic letters
    arabic_letters = [u"\u0627",u"\u0628",u"\u062A",u"\u062B",u"\u062C",u"\u062D",u"\u062E",u"\u062F",u"\u0630",u"\u0631",u"\u0632",u"\u0633",u"\u0634",u"\u0635",u"\u0636",u"\u0637",u"\u0638",u"\u0639",u"\u063A",u"\u0641",u"\u0642",u"\u0643",u"\u0644",u"\u0645",u"\u0646",u"\u0647",u"\u0648",u"\u064A"]
    for letter in arabic_letters:
        x = reduceString(x, letter)
    
    # remove maddah
    x = x.replace(u'\u0640','')
    
    # ال + لا
    x = reduceString(x, u'\u0627\u0644')
    x = reduceString(x, u'\u0644\u0627')
        
    x = reduceString(x, "a")
    x = reduceString(x, "s")
    x = reduceString(x, "h")
    x = reduceString(x, "o")
    x = reduceString(x, "y")
    
    x = reduceString(x, "_")
    
    return x
    
def cleanAll(X):
    # perform clean on all elements
    clean_x = []
    for x in X:
        clean_x.append(clean(x))
    return clean_x

def listFeatures(X):
    # list the features of the tfidf vectorizer
    vectorizer = TfidfVectorizer()
    vect = vectorizer.fit_transform(X)
    vocab = []
    for voc in vectorizer.vocabulary_:
        vocab.append(voc)
    vocab.sort()
    print(len(vocab))
    print("\n")
    for voc in vocab:
        print voc+"\n"


In [166]:
'''    main    '''
df = read_csv('corpusCinema.csv', engine='python', encoding="UTF-8")
array =  df.values

X = array[:, 0] #Tweet
Y = array[:, 1] #Sentiment

for n,i in enumerate(Y):
    if i==1:
        Y[n]="Positive"
    elif i==0:
        Y[n]="Neutral"
    elif i==-1:
        Y[n]="Negative"

X = cleanAll(X)
listFeatures(X)
    
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=7)

16397


0105

0150

017

050

10

100

1000

100الف

100مليون

1010

10الاف

10ايام

10سنوات

10كيلو

10هلات

11

115

12

120

12د

13

130

130000

14

1439

1439ه

15

150

150260

150ريال

15959

15دقيقه

15سنه

16

160

1606غ

16ديسمبر28ربيع

17

18

180

18122017

18هله

19

1905

1919

1919م

1948

1950

1950م

1956م

1967م

1974

1976

1976م

1977م

1979

1995

1اذا

1التزام

1براد

1عرض

20

200

2000

2002

2007

2008

2009

20100

2011

2012

2014

2015

2016

2016م

2017

2017خوش

2017عام_المفاجات

2017م

2018

20180101

2020

2030

208

2090م

20الف

21

212

21_ديسمبر_صرف_حساب_المواطن

21جماداول

230

2300

24

24س

25

250

26

2620

280

2840

2اما

2بقيمه

2تريليون

2جيسون

2ربيع

30

300

300ريال

30_ثانيه

31

33603138

338

34

35

360

3900

3d

3توم

3ساعاتالدعايه

3شباب

3طبعا

3عرض

40

400

4000

40عاما

430

45

467رياليبحث

49

4_بروس

4d

4k

4ايام

50

500

50603048

50الف

50ريال

50سينما

530

55

560مليون

5ريال

60

600

6000

600ريال

61

64

65

6548


بدبي

بدت

بدخلها

بدخول

بدخوله

بدر

بدري

بدريه

بدعم

بدقايق

بدقه

بدل

بدلا

بدلات

بدمعهلكن

بدوا

بدور

بدوري

بدولتنا

بدون

بدوي

بديت

بديرتكم

بدينا

بديهيات

بذا

بذريعه

بذكري

بذل

بذلك

بذله

بذنوبنا

بذهاب

بذيك

برا

براءه

براتب

براحتك

براحتي

براق

برامج

برامجها

برايتون

برايك

برب

بربع

بربكم

برتبه

برتويت

برجع

برحمتك

برد

برداومريضاين

برده

بردود

برذرز

برساله

برسايل

برشلونه_ديبورتيفو

برضو

برضوا

برعايه

برغم

برفقه

بركاتك

بركب

بركه

برنامج

بره

بروابط

بروجكتر

بروح

بروحيه

بروشور

برومو

برويتك

بري

بريء

برياسه

بريده

بريزون

بريطانيا

بريك

بزباله

بزبوز

بزعمه

بزمن

بزوغ

بزياده

بزيارتهم

بس

بس_اقول

بساطه

بساعتين

بسب

بسبه

بسبهم

بستخدمها

بسترك

بسحر

بسدادها

بسرعه

بسروال

بسريه

بسريهالاضافه

بسطه

بسعر

بسعرها

بسعه

بسعيده

بسفر

بسكاكا

بسكته

بسلام

بسم

بسنما

بسنماء

بسنه

بسواعد

بسي

بسيارتي

بسياره

بسيط

بسيطه

بسينما

بشار_الاسد

بشارالاسد

بشارع

بشاشات

بشاشتكم

بشاشه

بشان

بشبك

بشتري

بشده

بشر

بشروط

بشري

بش


عالقطات

عالم

عالمقطع

عالمك

عالمي

عالميا

عالمياوعن

عالميه

عاله

عالي

عاليسار

عاليه

عام

عاما

عامل

عامله

عامه

عامين_لتاسيس_الجيش_السلماني

عانا

عاهره

عاوز

عاوزين

عايديه

عايش

عايشت

عايشين

عايض

عايلتنا

عايله

عايليه

عباره

عبايات

عبايه

عبد

عبد_اله_السناني

عبدالرازق

عبدالرحمن

عبدالرزاق

عبدالعزير

عبدالعزيز

عبدالعزيزاله

عبدالقادر

عبدالمحسن

عبداله

عبداله_بن_جليغم

عبداله_بن_زايد

عبده

عبر

عبور

عبيد

عبيد_ما_خذا_فسحته

عتباريه

عتمه

عثمان

عثمين

عثيمين

عجبا

عجبتني

عجزنا

عجل

عجلان

عجله

عجلو

عجلوا

عجلينبه

عجيب

عد

عدا

عدسات_الاسنان

عدل

عدله

عدم

عدن

عدنان

عده

عدي

عديده

عذاب

عذابك

عذر

عذرك

عذركم

عذرنا

عذرهم

عذري

عرب_نت_الرياض

عربي

عربي21

عربي_واقليمي

عربيا

عربيه

عرس

عرض

عرضت

عرضنا

عرضه

عرضها

عرعر

عرفنا

عروض

عروض_الرياض

عروض_العثيم_نهايه_العام

عروض_جولي_شيك_12_12

عروض_نهايه_العام

عروضكم

عروضه

عروقه

عريشي

عريض

عريكه

عز

عزاء

عزاب

عزاي

عزل

عزل_اسطح_خزانات

عزمي

عزيز

عزيزاتي

عزيزي

عزيزي_المواطن

ع

نبض

نبضات_حرف_لدعم

نبع

نبغا

نبغاه

نبغاها

نبغي

نبل

نبلوهم

نبي

نبيع

نبيك

نبينا

نبينامحمد

نبيه

نبيها

نت

نتابع

نتابعه

نتابعها

نتايج

نتبع

نتجه

نتحج

نتحدث

نتحمس

نتخلص

نتخلي

نتصادم

نتصر

نتضارب

نتطلع

نتطنز

نتطور

نتظر

نتظره

نتعامل

نتعلم

نتفاجاء

نتفاعل

نتفرج

نتفرجها

نتفلكس

نتفليكس

نتمني

نتناقش

نتنياهو

نتواعد

نتوظف

نثق

نجاحه

نجاحهم

نجبر

نجد

نجران

نجرب

نجرهار

نجعل

نجم

نجم_الترند_طالب_الزهراني100k

نجمكل

نجمنا

نجود

نجوم

نجوي

نجيب

نجيكم

نحارب

نحب

نحبه

نحتاج

نحتري

نحجز

نحرهم

نحضر

نحط

نحكم

نحمل

نحن

نحنا

نحو

نخاف

نختار

نخرج

نخف

نخليها

نخنخ

نداوم

ندخل

ندري

ندعث

ندم

ندور

ندوه

نرابط

نراعي

نراه

نراها

نرتاح

نرجع

نرفض

نرفع

نركز

نروح

نري

نريد

نريد_بارتيات_مختلطه_بالسعوديه

نريده

نريدها

نزداد

نزع

نزل

نزلو

نزول

نزوله

نزيلهم

نسافر

نسال

نسال_اله_حسن_الخاتمه

نسالك

نسايه

نسب

نسبه

نستاهلها

نستخدم

نستخدمها

نسترجع

نستشير

نستطيع

نستغل

نستغلها

نستفيد

نستلهم

نستني

نستورد

نسجل

نسجلك

نسخه




يقصد

يقصدون

يقصقصوها

يقطع

يقطعون

يقعد

يقفز

يقفلون

يقفلونها

يقفلوه

يقل

يقلبونها

يقلقني

يقلك

يقلون

يقهر

يقود

يقودها

يقول

يقولك

يقولون

يقولي

يقوم

يقومون

يقوي

يقينا

يكت

يكتب

يكتبون

يكتفون

يكثرون

يكثفون

يكذب

يكرهون

يكسرك

يكشف

يكفي

يكفينا

يكلمني

يكمله

يكملوا

يكمن

يكن

يكون

يلا

يلاحقنا

يلاقي

يلبيه

يلحقنا

يلد

يلزم

يلزمك

يلزمهوش

يلعبون

يلعن

يلغون

يلفت

يلقاه

يلقون

يلقي

يله

يلهون

يلهي

يليق

يم

يمارسن

يماه

يمتص

يمتع

يمتلك

يمتلكون

يمتنع

يمثل

يمثلني

يمثلون

يمخانيث

يمدح

يمدحون

يمدير

يمدينا

يمديني

يمديها

يمديهم

يمر

يمسك

يمسكونه

يمشوا

يمشون

يمكرون

يمكن

يمكنا

يمكنك

يملك

يملوه

يمنع

يمنعون

يمنيه

يمه

يمهل

يموت

يموتون

يميل

يمين

يمينا

ينا

يناسب

يناسبنا

يناظر

ينافس

يناقش

يناقشونها

يناهق

ينبت

ينبح

ينبسط

ينبغي

ينتبهون

ينتج

ينتضرون

ينتظر

ينتظرنا

ينتظرون

ينتفي

ينتقدنا

ينتقدون

ينتهب

ينتهي

ينجح

ينحصر

ينزل

ينساها

ينسب

ينسوا

ينسي

ينسينا

ينشر

ينصح

ينطبق

ينطح

ينطق

ينطم

ينظر

ينعاد


In [173]:
pipeline = Pipeline([
    ('1', TfidfVectorizer()),
    ('2', TfidfTransformer()),
    ('3', SVC(kernel='linear',random_state=7,decision_function_shape='ovo'))
])
parameters = {
#     '1__max_df': (0.5, 1.0, 1.5),
    
#     '2__use_idf': (True, False),
#     '2__norm': ('l1', 'l2'),
    
    '3__C': (0.1, 1.0, 10),
    '3__tol': (1e-4,  1e-6, 1e-8),
    #'3__class_weight': (None, 'balanced'),
    '3__degree': ( 1,3),
    
}

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)
t0 = time()
grid_search.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
printBestParameter(grid_search)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   49.9s
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:  1.0min finished


done in 65.794s
Best score: 0.753
Best parameters set:
	3__C: 1.0
	3__degree: 1
	3__tol: 0.0001


In [174]:
pipeline = Pipeline([
    ('1', TfidfVectorizer()),
    ('2', TfidfTransformer()),
    ('3', MultinomialNB(random_state=7))
])
parameters = {
    '1__max_df': (0.25, 0.5, 0.75, 1.0),
    
    '2__use_idf': (True, False),
    '2__norm': ('l1', 'l2'),
    
    '3__alpha': (1.0e-10, 1, 10),
}

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)
t0 = time()
grid_search.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")

printBestParameter(grid_search)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.0s


done in 13.337s
Best score: 0.722
Best parameters set:
	1__max_df: 0.5
	2__norm: 'l1'
	2__use_idf: False
	3__alpha: 1e-10


[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed:   13.1s finished


In [ ]:
pipeline = Pipeline([
    ('1', TfidfVectorizer()),
    ('2', TfidfTransformer()),
    ('3', RandomForestClassifier(random_state=7))
])
parameters = {
#     '1__max_df': (0.25, 0.5, 0.75, 1.0), # 1.0
    '1__use_idf': (True, False), # true
    '1__norm': ('l1', 'l2'), # l2
    
    '2__use_idf': (True, False), # true
    '2__norm': ('l1', 'l2'), # l2
    
    '3__n_estimators': (5, 10, 15, 20, 25, 35, 50), # 10
    '3__criterion': ('gini', 'entropy'), # gini
    '3__class_weight': (None, 'balanced'), # None
}

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)
t0 = time()
grid_search.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")

printBestParameter(grid_search)

Fitting 3 folds for each of 448 candidates, totalling 1344 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  2.9min


In [176]:
pipeline = Pipeline([
    ('1', TfidfVectorizer()),
    ('2', TfidfTransformer()),
    ('3', LogisticRegression())
])
parameters = {
    '1__max_df': (0.5, 0.75, 1.0),
    
    '2__use_idf': (True, False),
    '2__norm': ('l1', 'l2'),
}

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)
t0 = time()
grid_search.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")

printBestParameter(grid_search)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    4.1s finished


done in 4.472s
Best score: 0.724
Best parameters set:
	1__max_df: 1.0
	2__norm: 'l2'
	2__use_idf: False
